## Implementacja metody UTAstar dyskretnej

In [1]:
import numpy as np
from scipy.interpolate import interp1d

# Funkcja do wyznaczania funkcji użyteczności dla wersji dyskretnej
def UTA_function_discrete(values, reference_points, lambda_param=1):
    """
    Aproksymuje funkcje użyteczności dla wersji dyskretnej.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych (najlepszy, najgorszy, neutralny)
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Lista funkcji użyteczności dla wszystkich alternatyw
    """
    utility_values = []
    
    # Iterujemy przez alternatywy
    for val in values:
        alternative_utility = []
        
        # Iterujemy przez każde kryterium
        for i in range(len(val)):
            ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
            # Interpolacja liniowa dla każdego kryterium
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_value = interp_func(val[i]) * lambda_param  # Użycie λ
            alternative_utility.append(utility_value)
        
        utility_values.append(alternative_utility)
    
    return np.array(utility_values)

# Funkcja do porównania alternatyw (dyskretna wersja)
def UTAstar_discrete(values, reference_points, lambda_param=1):
    """
    Metoda UTA Star dla wersji dyskretnej (wielokryterialnej) uwzględniająca preferencje decydenta.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_discrete(values, reference_points, lambda_param)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution_idx = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution_idx, utility_values

# Przykład użycia (wersja dyskretna)
values_discrete = np.array([[20, 30], [50, 60], [80, 90], [35, 45], [65, 70]])  # Przykładowe wartości alternatyw
reference_points_discrete = np.array([[20, 30], [80, 90], [50, 60]])  # Punkty referencyjne (najlepszy, najgorszy, neutralny)
lambda_param = 1.5  # Przykładowy współczynnik λ (preferencje decydenta)
best_solution_discrete_idx, utility_values_discrete = UTAstar_discrete(values_discrete, reference_points_discrete, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (dyskretna): {values_discrete[best_solution_discrete_idx]} z funkcją użyteczności {utility_values_discrete[best_solution_discrete_idx]}")


Najlepsza alternatywa (dyskretna): [80 90] z funkcją użyteczności [1.5 1.5]


## Implementacja UTAstar ciągłej

In [2]:
import numpy as np
from scipy.interpolate import interp1d

# Funkcja do wyznaczania funkcji użyteczności (ciągła wersja z preferencjami)
def UTA_function_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Aproksymuje funkcje użyteczności w przestrzeni ciągłej dla wielu kryteriów.
    Uwzględnia preferencje decydenta przez współczynnik λ i punkty referencyjne.
    
    :param values: Lista wartości kryteriów dla alternatyw
    :param reference_points: Lista punktów referencyjnych (np. [najlepszy, najgorszy, neutralny])
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Funkcje użyteczności
    """
    utility_values = []
    for i in range(len(reference_points[0])):  # Iterujemy po kryteriach
        ref_values = [ref[i] for ref in reference_points]  # Zbiór punktów referencyjnych dla kryterium i
        if use_polynomial:
            # Przy użyciu funkcji aproksymacji wielomianowej
            coef = np.polyfit(ref_values, [0, 1, 0], 2)  # Przykład: funkcja kwadratowa
            poly_func = np.poly1d(coef)
            utility_values.append(poly_func(values[:, i]) * lambda_param)  # Użycie λ
        else:
            # Interpolacja liniowa
            interp_func = interp1d(ref_values, [0, 1, 0], kind='linear', fill_value="extrapolate")
            utility_values.append(interp_func(values[:, i]) * lambda_param)  # Użycie λ
    return np.array(utility_values).T  # Zwracamy funkcje użyteczności dla wszystkich alternatyw

# Funkcja do porównania alternatyw (ciągła wersja z uwzględnieniem preferencji)
def UTAstar_continuous(values, reference_points, lambda_param=1, use_polynomial=False):
    """
    Metoda UTA Star dla wersji ciągłej (wielokryterialnej) uwzględniająca preferencje decydenta.
    
    :param values: Lista wartości alternatyw w przestrzeni kryteriów
    :param reference_points: Lista punktów referencyjnych dla każdego kryterium
    :param lambda_param: Współczynnik λ (preferencje decydenta)
    :param use_polynomial: Jeśli True, stosuje aproksymację wielomianową, w przeciwnym razie interpolację
    :return: Wybrana alternatywa
    """
    # Wyznaczanie funkcji użyteczności
    utility_values = UTA_function_continuous(values, reference_points, lambda_param, use_polynomial)
    
    # Wybór rozwiązania kompromisowego - wybieramy alternatywę z maksymalną sumą użyteczności
    summed_utility = np.sum(utility_values, axis=1)
    best_solution = np.argmax(summed_utility)  # Wybieramy alternatywę z największą sumą funkcji użyteczności
    return best_solution, utility_values

# Przykład użycia (ciągła wersja)
values_continuous = np.random.uniform(0, 100, (10, 2))  # Przykładowe wartości alternatyw w przestrzeni 2 kryteriów
reference_points_continuous = np.array([[20, 30], [50, 60], [80, 90]])  # Punkty referencyjne w przestrzeni 2 kryteriów
lambda_param = 1.5  # Przykładowy współczynnik λ (preferencje decydenta)
best_solution_continuous, utility_values_continuous = UTAstar_continuous(values_continuous, reference_points_continuous, lambda_param=lambda_param)
print(f"Najlepsza alternatywa (ciągła): {values_continuous[best_solution_continuous]} z funkcją użyteczności {utility_values_continuous[best_solution_continuous]}")


Najlepsza alternatywa (ciągła): [43.79016634 69.04219988] z funkcją użyteczności [1.18950832 1.04789001]


# Testy obliczeniowe

## Testy Dyskretne

### Dyskretne 3-wymiarowe

In [3]:
# Przykładowe wartości alternatyw w 3 wymiarach (3 kryteria)
values_discrete_3d = np.array([
    [20, 30, 40],
    [50, 60, 70],
    [80, 90, 100],
    [35, 45, 55],
    [65, 75, 85]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 3 wymiarach
reference_points_discrete_3d = np.array([
    [20, 30, 40],  # Najlepszy
    [80, 90, 100],  # Najgorszy
    [50, 60, 70]   # Neutralny
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji dyskretnej
best_solution_discrete_3d_idx, utility_values_discrete_3d = UTAstar_discrete(values_discrete_3d, reference_points_discrete_3d, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (dyskretna, 3D): {values_discrete_3d[best_solution_discrete_3d_idx]} z funkcją użyteczności {utility_values_discrete_3d[best_solution_discrete_3d_idx]}")

Najlepsza alternatywa (dyskretna, 3D): [ 80  90 100] z funkcją użyteczności [1.5 1.5 1.5]


### Dyskretne 4-wymiarowe

In [4]:
# Przykładowe wartości alternatyw w 4 wymiarach (4 kryteria)
values_discrete_4d = np.array([
    [20, 30, 40, 50],
    [50, 60, 70, 80],
    [80, 90, 100, 110],
    [35, 45, 55, 65],
    [65, 75, 85, 95]
])

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_discrete_4d = np.array([
    [20, 30, 40, 50],  # Najlepszy
    [80, 90, 100, 110],  # Najgorszy
    [50, 60, 70, 80]   # Neutralny
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji dyskretnej
best_solution_discrete_4d_idx, utility_values_discrete_4d = UTAstar_discrete(values_discrete_4d, reference_points_discrete_4d, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (dyskretna, 4D): {values_discrete_4d[best_solution_discrete_4d_idx]} z funkcją użyteczności {utility_values_discrete_4d[best_solution_discrete_4d_idx]}")


Najlepsza alternatywa (dyskretna, 4D): [ 80  90 100 110] z funkcją użyteczności [1.5 1.5 1.5 1.5]


## Testy Ciągłe

### Ciągłe 3-wymiarowe

In [5]:
# Przykładowe wartości alternatyw w 3 wymiarach (3 kryteria) w przestrzeni ciągłej
values_continuous_3d = np.random.uniform(0, 100, (10, 3))  # 10 alternatyw, 3 kryteria

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 3 wymiarach
reference_points_continuous_3d = np.array([
    [20, 30, 40],  # Najlepszy
    [50, 60, 70],  # Neutralny
    [80, 90, 100]  # Najgorszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji ciągłej
best_solution_continuous_3d, utility_values_continuous_3d = UTAstar_continuous(values_continuous_3d, reference_points_continuous_3d, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (ciągła, 3D): {values_continuous_3d[best_solution_continuous_3d]} z funkcją użyteczności {utility_values_continuous_3d[best_solution_continuous_3d]}")


Najlepsza alternatywa (ciągła, 3D): [43.21727356 81.5128798  69.44270757] z funkcją użyteczności [1.16086368 0.42435601 1.47213538]


### Ciągłe 4-wymiarowe

In [6]:
# Przykładowe wartości alternatyw w 4 wymiarach (4 kryteria) w przestrzeni ciągłej
values_continuous_4d = np.random.uniform(0, 100, (10, 4))  # 10 alternatyw, 4 kryteria

# Punkty referencyjne (najlepszy, najgorszy, neutralny) w 4 wymiarach
reference_points_continuous_4d = np.array([
    [20, 30, 40, 50],  # Najlepszy
    [50, 60, 70, 80],  # Neutralny
    [80, 90, 100, 110]  # Najgorszy
])

lambda_param = 1.5  # Współczynnik λ (preferencje decydenta)

# Wywołanie metody UTA Star dla wersji ciągłej
best_solution_continuous_4d, utility_values_continuous_4d = UTAstar_continuous(values_continuous_4d, reference_points_continuous_4d, lambda_param=lambda_param)

print(f"Najlepsza alternatywa (ciągła, 4D): {values_continuous_4d[best_solution_continuous_4d]} z funkcją użyteczności {utility_values_continuous_4d[best_solution_continuous_4d]}")


Najlepsza alternatywa (ciągła, 4D): [68.56251629 42.51116159 85.89683855 95.70861633] z funkcją użyteczności [0.57187419 0.62555808 0.70515807 0.71456918]
